### Adnan Altukleh, Abdulkarim Dawalibi
### Course DV2627

### Importing necessary libraries

In [12]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report,confusion_matrix
from scipy.stats import skew, kurtosis
from scipy.signal import welch
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from scipy.stats import entropy

### Reading the data

In [13]:
train_oper=pd.read_csv("train_operational_readouts.csv")
train_tte=pd.read_csv("train_tte.csv")

In [14]:
# function to calculate the time to event
def set_class_label(row):
    if row['in_study_repair'] == 0:
        return 0
    elif 0 <= row['time'] <= 6:
        return 4
    elif 6 <= row['time'] <= 12:   
        return 3
    elif 12 <= row['time'] <= 24:
        return 2
    elif 24 <= row['time'] <= 48:
        return 1
    elif row['time'] > 48:
        return 1
    else:
        raise ValueError("Invalid time value: {}".format(row['time']))

In [15]:
# Get the last row for each vehicle_id in train_oper
last_rows = train_oper.groupby('vehicle_id').tail(1)

# Merge train_tte with last_rows on vehicle_id
train_oper_label = pd.merge(last_rows, train_tte, on='vehicle_id')

# Calculate the difference between time_step and length_of_study_time_step
train_oper_label['time'] = train_oper_label['length_of_study_time_step'] - train_oper_label['time_step']
train_oper_label

,vehicle_id,time_step,171_0,666_0,427_0,837_0,167_0,167_1,167_2,167_3,...,397_29,397_30,397_31,397_32,397_33,397_34,397_35,length_of_study_time_step,in_study_repair,time
0,0,507.4,10189950.0,372685.0,NaN,41670.0,NaN,NaN,NaN,NaN,...,367.0,36431.0,3534852.0,4290245.0,1027186.0,138939.0,792.0,510.0,0,2.6
1,2,281.0,5648790.0,289371.0,233978571.0,68717.0,10415.0,9137870.0,74655621.0,45991626.0,...,246.0,18151.0,1193941.0,1680728.0,328910.0,57511.0,793.0,281.8,0,0.8
2,3,291.2,7603590.0,230831.0,273813365.0,100121.0,5918.0,8225139.0,17004223.0,10504195.0,...,204.0,7952.0,719693.0,2648967.0,531790.0,76915.0,712.0,293.4,0,2.2
3,4,203.0,4842780.0,210381.0,174662976.0,152385.0,7128.0,4342398.0,13348382.0,11538870.0,...,742.0,10529.0,370756.0,1651890.0,1282530.0,453350.0,1962.0,210.0,0,7.0
4,5,357.6,6623040.0,280531.0,246994778.0,164673.0,NaN,NaN,NaN,NaN,...,2601.0,9754.0,465567.0,1684524.0,979654.0,515597.0,8663.0,360.4,0,2.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23545,33639,112.6,2894100.0,75859.0,98975802.0,10040.0,6631.0,5229663.0,10519563.0,8797533.0,...,156.0,7958.0,239587.0,1024221.0,384568.0,101945.0,401.0,137.4,0,24.8
23546,33640,123.2,3925455.0,138179.0,139017335.0,35726.0,15397.0,5757292.0,11640261.0,7568543.0,...,556.0,26526.0,409625.0,1390447.0,474318.0,338311.0,5127.0,124.6,0,1.4
23547,33641,116.0,2567085.0,118371.0,91726867.0,9122.0,6783.0,3383972.0,6551850.0,6232055.0,...,100.0,25511.0,796852.0,1395692.0,264116.0,33488.0,222.0,123.0,0,7.0
23548,33642,126.4,3350700.0,80078.0,118733863.0,11251.0,755.0,9392926.0,16152271.0,7956861.0,...,80.0,16320.0,910530.0,1851490.0,290285.0,41111.0,536.0,126.6,0,0.2


In [16]:
train_oper_label['class_label'] = train_oper_label.apply(set_class_label, axis=1)

In [17]:
tt=train_oper_label.drop(columns=train_oper_label.columns[1:-1],axis=1)
training_data=pd.merge(tt,train_oper,on='vehicle_id')

In [18]:
training_data

,vehicle_id,class_label,time_step,171_0,666_0,427_0,837_0,167_0,167_1,167_2,...,397_26,397_27,397_28,397_29,397_30,397_31,397_32,397_33,397_34,397_35
0,0,0,11.2,167985.0,10787.0,7413813.0,2296.0,4110.0,1296420.0,1628265.0,...,95728.0,15609.0,1984.0,8.0,784.0,150228.0,261904.0,93172.0,17874.0,452.0
1,0,0,11.4,167985.0,10787.0,7413813.0,2296.0,4111.0,1302855.0,1628265.0,...,95729.0,15610.0,1984.0,8.0,784.0,150228.0,261905.0,93172.0,17874.0,452.0
2,0,0,19.6,331635.0,14525.0,13683604.0,2600.0,NaN,NaN,NaN,...,142900.0,19263.0,2441.0,12.0,1420.0,204832.0,313485.0,106464.0,19306.0,452.0
3,0,0,20.2,354975.0,15015.0,14540449.0,2616.0,NaN,NaN,NaN,...,150565.0,19832.0,2522.0,12.0,1444.0,211688.0,318901.0,107745.0,19406.0,453.0
4,0,0,21.0,365550.0,15295.0,14966985.0,2720.0,NaN,NaN,NaN,...,155913.0,20573.0,2562.0,12.0,1445.0,213956.0,323997.0,109514.0,19535.0,454.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122447,33643,0,101.0,2136810.0,41412.0,81068654.0,10365.0,843.0,6408272.0,10964376.0,...,727848.0,34448.0,6540.0,36.0,19500.0,612343.0,626033.0,100155.0,17033.0,24.0
1122448,33643,0,107.0,2283285.0,44660.0,86639835.0,12317.0,843.0,6596477.0,12358026.0,...,780576.0,37488.0,6964.0,40.0,20484.0,652688.0,670517.0,107367.0,18901.0,24.0
1122449,33643,0,113.8,2462865.0,48370.0,93439419.0,13910.0,843.0,6669542.0,12830421.0,...,837953.0,40050.0,7524.0,44.0,21688.0,698824.0,722453.0,115851.0,21237.0,28.0
1122450,33643,0,119.8,2627250.0,51101.0,99682931.0,14854.0,843.0,7074468.0,13604706.0,...,898070.0,42866.0,7992.0,48.0,22732.0,757856.0,777630.0,123155.0,22357.0,32.0


### Selecting Specific number of vehicles form each class

In [19]:
# selecting the vehicels from class 4 and 0 so that have the same number of vehicles as class 3 other classes have all vehicels
class_1 = training_data.loc[(training_data["class_label"] == 1)]["vehicle_id"].unique()
class_3 = training_data.loc[(training_data["class_label"] == 3)]["vehicle_id"].unique()
class_2 = training_data.loc[(training_data["class_label"] == 2)]["vehicle_id"].unique()
class_4 = training_data.loc[(training_data["class_label"] == 4)&(training_data["vehicle_id"].isin(training_data.groupby("vehicle_id").filter(lambda x: x.notnull().all().all())["vehicle_id"].unique()))]["vehicle_id"].unique()[:len(class_3)]
class_0 = training_data.loc[(training_data["class_label"] == 0)&(training_data["vehicle_id"].isin(training_data.groupby("vehicle_id").filter(lambda x: x.notnull().all().all())["vehicle_id"].unique()))]["vehicle_id"].unique()[:len(class_3)]

df = pd.concat([
    training_data.loc[training_data["vehicle_id"].isin(class_1)],
    training_data.loc[training_data["vehicle_id"].isin(class_2)],
    training_data.loc[training_data["vehicle_id"].isin(class_3)],
    training_data.loc[training_data["vehicle_id"].isin(class_4)],
    training_data.loc[training_data["vehicle_id"].isin(class_0)]
])
df

,vehicle_id,class_label,time_step,171_0,666_0,427_0,837_0,167_0,167_1,167_2,...,397_26,397_27,397_28,397_29,397_30,397_31,397_32,397_33,397_34,397_35
28407,457,1,4.0,152010.0,4620.0,5721991.0,840.0,870.0,292800.0,264601.0,...,47421.0,4104.0,684.0,8.0,440.0,14764.0,44472.0,13781.0,1300.0,88.0
28408,457,1,6.4,216975.0,6307.0,8140374.0,968.0,870.0,347955.0,352906.0,...,67685.0,5656.0,856.0,8.0,504.0,19084.0,58753.0,18129.0,1585.0,88.0
28409,457,1,7.4,240930.0,7259.0,9056223.0,1040.0,870.0,365775.0,391186.0,...,74121.0,6216.0,908.0,8.0,600.0,21800.0,66422.0,20209.0,1717.0,88.0
28410,457,1,12.4,411105.0,14742.0,15511024.0,3097.0,870.0,519180.0,588331.0,...,115125.0,9429.0,1409.0,8.0,984.0,41621.0,106886.0,29465.0,2769.0,88.0
28411,457,1,16.2,536340.0,19376.0,20120739.0,4641.0,870.0,605685.0,708646.0,...,140869.0,11737.0,1741.0,8.0,1300.0,53785.0,130499.0,34993.0,3434.0,92.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26530,419,0,255.0,5745315.0,176567.0,255020679.0,34884.0,12483.0,10757290.0,44515805.0,...,3054650.0,211788.0,21769.0,100.0,40226.0,858341.0,1270497.0,310173.0,33379.0,368.0
26531,419,0,256.6,5781000.0,178128.0,256745534.0,35764.0,12483.0,10814590.0,44872475.0,...,3078454.0,213320.0,22037.0,120.0,40462.0,868733.0,1286281.0,312729.0,33775.0,392.0
26532,419,0,262.0,5909760.0,182728.0,262519522.0,37460.0,12483.0,11009530.0,45788990.0,...,3142470.0,217289.0,22590.0,120.0,40895.0,892142.0,1325317.0,319898.0,34463.0,404.0
26533,419,0,270.4,6129885.0,191317.0,272078159.0,40116.0,12483.0,11425450.0,47487366.0,...,3260199.0,224521.0,23539.0,124.0,41971.0,938843.0,1399401.0,333210.0,35859.0,404.0


### Convert the data to power spectral density (PSD)

In [20]:
# function to estimate the sampling frequency
def estimate_fs(group):
    if len(group) < 2:
        return np.nan
    time_diffs = group['time_step'].diff().dropna()
    mean_diff = time_diffs.mean()
    if mean_diff == 0:
        return np.nan
    return 1 / mean_diff
# function to calculate the power spectral density
def compute_psd(group):

    # Estimate the sampling frequency for the group
    fs = estimate_fs(group)
    
    # Columns to skip
    columns_to_skip = ['vehicle_id', 'class_label', 'time_step']
    
    # Prepare a dictionary to collect PSD data
    psd_data = {'vehicle_id': group['vehicle_id'].iloc[0]}
    
    # Iterate over columns that are not in columns_to_skip
    for column in [col for col in group.columns if col not in columns_to_skip]:
        # Calculate the PSD using Welch's method
        freqs, Pxx_den = welch(group[column], fs=fs)
        psd_data[f'{column}_psd'] = Pxx_den
        psd_data[f'{column}_freqs'] = freqs 
    
    # Create a DataFrame for the current vehicle's PSD data
    psd_df = pd.DataFrame(psd_data)
    
    return psd_df

In [21]:
# Group the training data by 'vehicle_id' and apply the compute_psd function
grouped = df.groupby('vehicle_id')
grouped_psd_dfs = grouped.apply(compute_psd)
psd_combined_df = grouped_psd_dfs.reset_index(drop=True)

c:\Users\adnan\anaconda3\envs\new\lib\site-packages\scipy\signal\_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 33, using nperseg = 33
  warnings.warn('nperseg = {0:d} is greater than input length '
c:\Users\adnan\anaconda3\envs\new\lib\site-packages\scipy\signal\_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 42, using nperseg = 42
  warnings.warn('nperseg = {0:d} is greater than input length '
c:\Users\adnan\anaconda3\envs\new\lib\site-packages\scipy\signal\_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 45, using nperseg = 45
  warnings.warn('nperseg = {0:d} is greater than input length '
c:\Users\adnan\anaconda3\envs\new\lib\site-packages\scipy\signal\_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 94, using nperseg = 94
  warnings.warn('nperseg = {0:d} is greater than input length '
c:\Users\adnan\anaconda3\envs\new\lib\site-packages\scipy\signal\_sp

In [22]:
# label the data
Label=train_oper_label.drop(columns=train_oper_label.columns[1:-1],axis=1)
psd_combined_df=pd.merge(Label,psd_combined_df,on='vehicle_id')
psd_combined_df

,vehicle_id,class_label,171_0_psd,171_0_freqs,666_0_psd,666_0_freqs,427_0_psd,427_0_freqs,837_0_psd,837_0_freqs,...,397_31_psd,397_31_freqs,397_32_psd,397_32_freqs,397_33_psd,397_33_freqs,397_34_psd,397_34_freqs,397_35_psd,397_35_freqs
0,2,0,1.801266e+13,0.000000,8.311010e+10,0.000000,3.271568e+16,0.000000,3.222816e+09,0.000000,...,9.774308e+11,0.000000,2.064357e+12,0.000000,1.198911e+11,0.000000,4.901280e+09,0.000000,7.737257e+05,0.000000
1,2,0,1.368028e+14,0.003468,3.360947e+11,0.003468,2.362335e+17,0.003468,1.444445e+10,0.003468,...,6.832272e+12,0.003468,1.211736e+13,0.003468,3.960113e+11,0.003468,7.729815e+09,0.003468,8.025761e+05,0.003468
2,2,0,5.985169e+12,0.006936,1.305118e+10,0.006936,1.028956e+16,0.006936,7.871946e+08,0.006936,...,3.589297e+11,0.006936,5.813196e+11,0.006936,1.255181e+10,0.006936,6.134489e+07,0.006936,3.745853e+04,0.006936
3,2,0,1.472598e+11,0.010404,8.890956e+08,0.010404,2.570168e+14,0.010404,1.398372e+08,0.010404,...,6.628615e+09,0.010404,5.238378e+10,0.010404,1.386922e+09,0.010404,1.855831e+07,0.010404,2.141204e+04,0.010404
4,2,0,4.223230e+11,0.013873,1.274794e+09,0.013873,7.235092e+14,0.013873,1.556801e+08,0.013873,...,1.190469e+10,0.013873,6.331151e+10,0.013873,1.479111e+09,0.013873,9.713297e+06,0.013873,1.217584e+03,0.013873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17287,33322,1,9.541001e+09,0.092406,7.023912e+05,0.092406,1.435532e+13,0.092406,2.154408e+05,0.092406,...,4.879930e+07,0.092406,7.012912e+07,0.092406,5.395848e+06,0.092406,3.035178e+05,0.092406,3.365770e+00,0.092406
17288,33322,1,1.129324e+09,0.110887,9.850954e+04,0.110887,1.597493e+12,0.110887,8.960493e+03,0.110887,...,9.384071e+06,0.110887,2.913123e+03,0.110887,1.035467e+05,0.110887,1.539612e+05,0.110887,1.610943e+00,0.110887
17289,33342,1,3.116126e+12,0.000000,8.025016e+10,0.000000,7.284098e+15,0.000000,4.263243e+09,0.000000,...,6.197137e+10,0.000000,1.874506e+12,0.000000,4.440485e+11,0.000000,7.011082e+11,0.000000,6.104641e+09,0.000000
17290,33342,1,1.779945e+13,0.009877,2.635761e+11,0.009877,3.726680e+16,0.009877,2.381335e+10,0.009877,...,3.617482e+11,0.009877,9.107484e+12,0.009877,1.572715e+12,0.009877,1.699218e+12,0.009877,1.436878e+10,0.009877


In [23]:
# fill the nan values with 0
psd_combined_df.fillna(0, inplace=True)

### Feature Extraction

In [24]:
def calculate_spectral_features(signal, fs=1.0):
    """
    Calculate spectral entropy and kurtosis of a signal.
    """
    f, Pxx = welch(signal, fs=fs, nperseg=min(256, len(signal)))
    psd_norm = Pxx / np.sum(Pxx)
    spectral_entropy = entropy(psd_norm)
    spectral_kurtosis = kurtosis(Pxx, fisher=True, bias=False)
    return spectral_entropy, spectral_kurtosis

def feature_extraction(df, sensor_columns):
    """
    Extract statistical and spectral features for each vehicle from multiple sensors.
    """
    # Initialize a list to hold the features for each vehicle
    features = []

    # Group the dataframe by vehicle_id
    grouped = df.groupby('vehicle_id')
    
    for vehicle_id, group in grouped:
        vehicle_features = [vehicle_id]
        
        for sensor in sensor_columns:
            sensor_data = group[sensor].values
            
            # Calculate statistical features
            mean = np.mean(sensor_data)
            median = np.median(sensor_data)
            std = np.std(sensor_data)
            var = np.var(sensor_data)
            minimum = np.min(sensor_data)
            maximum = np.max(sensor_data)
            skewness = skew(sensor_data)
            kurt = kurtosis(sensor_data)
            
            # Calculate spectral features
            spectral_entropy, spectral_kurtosis = calculate_spectral_features(sensor_data)
            
            # Append the features for this sensor to the vehicle features
            vehicle_features.extend([mean, median, std, var, minimum, maximum, skewness, kurt, spectral_entropy, spectral_kurtosis])
        
        # Append the features for this vehicle to the list
        features.append(vehicle_features)
    
    # Define column names dynamically based on the number of sensors
    column_names = ['vehicle_id']
    for sensor in sensor_columns:
        column_names.extend([f"{sensor}_mean", f"{sensor}_median", f"{sensor}_std", f"{sensor}_var", f"{sensor}_min", f"{sensor}_max", f"{sensor}_skewness", f"{sensor}_kurtosis", f"{sensor}_spectral_entropy", f"{sensor}_spectral_kurtosis"])
    
    # Create a DataFrame from the features
    feature_df = pd.DataFrame(features, columns=column_names)
    
    return feature_df

In [25]:
# feature extraction
feature_df = feature_extraction(psd_combined_df, psd_combined_df.columns[2:])
feature_df

C:\Users\adnan\AppData\Local\Temp\ipykernel_20464\2012294324.py:6: RuntimeWarning: invalid value encountered in divide
  psd_norm = Pxx / np.sum(Pxx)


,vehicle_id,171_0_psd_mean,171_0_psd_median,171_0_psd_std,171_0_psd_var,171_0_psd_min,171_0_psd_max,171_0_psd_skewness,171_0_psd_kurtosis,171_0_psd_spectral_entropy,...,397_35_freqs_mean,397_35_freqs_median,397_35_freqs_std,397_35_freqs_var,397_35_freqs_min,397_35_freqs_max,397_35_freqs_skewness,397_35_freqs_kurtosis,397_35_freqs_spectral_entropy,397_35_freqs_spectral_kurtosis
0,2,9.550564e+12,9.449861e+10,3.210733e+13,1.030881e+27,9.152348e+08,1.368028e+14,3.642203,11.520745,0.846027,...,0.027745,0.027745,0.016990,0.000289,0.0,0.055491,4.063478e-17,-1.208333,0.332223,8.928373
1,14,1.257518e+13,4.871166e+11,4.300053e+13,1.849046e+27,3.734490e+10,2.064344e+14,4.150941,15.784024,0.769561,...,0.023760,0.023760,0.014356,0.000206,0.0,0.047520,-1.301264e-16,-1.204969,0.294958,11.921489
2,25,5.541963e+13,3.561477e+11,1.928583e+14,3.719434e+28,8.872380e+10,9.363080e+14,4.139557,15.939487,0.741529,...,0.034101,0.034101,0.020564,0.000423,0.0,0.068203,-8.470090e-16,-1.204545,0.289702,11.922508
3,27,1.698739e+13,2.381277e+10,1.063952e+14,1.131994e+28,1.019048e+09,7.442551e+14,6.651192,42.483808,0.665104,...,0.068302,0.068302,0.040264,0.001621,0.0,0.136604,-1.384087e-16,-1.201042,0.237886,24.862938
4,28,5.152830e+12,6.470033e+09,3.021224e+13,9.127794e+26,3.278018e+07,2.169414e+14,6.755180,44.352183,0.658970,...,0.106497,0.106497,0.062704,0.003932,0.0,0.212995,-5.518674e-16,-1.200923,0.235689,25.858099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,32971,2.900359e+12,2.252884e+12,2.710656e+12,7.347656e+24,1.188604e+11,6.976806e+12,0.470994,-1.352791,0.659182,...,0.019599,0.019599,0.014608,0.000213,0.0,0.039199,-6.792461e-17,-1.360000,0.598270,-1.500000
590,33028,2.023403e+13,2.207573e+11,8.320764e+13,6.923511e+27,6.398269e+09,4.008166e+14,4.337765,16.896693,0.781639,...,0.036041,0.036041,0.021776,0.000474,0.0,0.072082,-5.667055e-16,-1.204969,0.294958,11.921489
591,33253,2.410189e+13,1.427981e+12,5.954394e+13,3.545481e+27,6.232435e+11,1.920250e+14,2.448100,4.045118,1.091072,...,0.020764,0.020764,0.013403,0.000180,0.0,0.041528,-7.817432e-17,-1.230000,0.478257,4.811490
592,33322,3.740236e+11,1.003053e+10,6.836876e+11,4.674287e+23,1.129324e+09,2.020392e+12,1.904838,1.851688,0.827700,...,0.055444,0.055444,0.036962,0.001366,0.0,0.110887,3.833919e-16,-1.250000,0.546863,3.546271


In [26]:
# label the data
label=train_oper_label.drop(columns=train_oper_label.columns[1:-1],axis=1)
data=pd.merge(label,feature_df,on='vehicle_id')

In [27]:
# fill the nan values with 0
data.fillna(0,inplace=True)
data

,vehicle_id,class_label,171_0_psd_mean,171_0_psd_median,171_0_psd_std,171_0_psd_var,171_0_psd_min,171_0_psd_max,171_0_psd_skewness,171_0_psd_kurtosis,...,397_35_freqs_mean,397_35_freqs_median,397_35_freqs_std,397_35_freqs_var,397_35_freqs_min,397_35_freqs_max,397_35_freqs_skewness,397_35_freqs_kurtosis,397_35_freqs_spectral_entropy,397_35_freqs_spectral_kurtosis
0,2,0,9.550564e+12,9.449861e+10,3.210733e+13,1.030881e+27,9.152348e+08,1.368028e+14,3.642203,11.520745,...,0.027745,0.027745,0.016990,0.000289,0.0,0.055491,4.063478e-17,-1.208333,0.332223,8.928373
1,14,0,1.257518e+13,4.871166e+11,4.300053e+13,1.849046e+27,3.734490e+10,2.064344e+14,4.150941,15.784024,...,0.023760,0.023760,0.014356,0.000206,0.0,0.047520,-1.301264e-16,-1.204969,0.294958,11.921489
2,25,0,5.541963e+13,3.561477e+11,1.928583e+14,3.719434e+28,8.872380e+10,9.363080e+14,4.139557,15.939487,...,0.034101,0.034101,0.020564,0.000423,0.0,0.068203,-8.470090e-16,-1.204545,0.289702,11.922508
3,27,4,1.698739e+13,2.381277e+10,1.063952e+14,1.131994e+28,1.019048e+09,7.442551e+14,6.651192,42.483808,...,0.068302,0.068302,0.040264,0.001621,0.0,0.136604,-1.384087e-16,-1.201042,0.237886,24.862938
4,28,0,5.152830e+12,6.470033e+09,3.021224e+13,9.127794e+26,3.278018e+07,2.169414e+14,6.755180,44.352183,...,0.106497,0.106497,0.062704,0.003932,0.0,0.212995,-5.518674e-16,-1.200923,0.235689,25.858099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,32971,2,2.900359e+12,2.252884e+12,2.710656e+12,7.347656e+24,1.188604e+11,6.976806e+12,0.470994,-1.352791,...,0.019599,0.019599,0.014608,0.000213,0.0,0.039199,-6.792461e-17,-1.360000,0.598270,-1.500000
590,33028,3,2.023403e+13,2.207573e+11,8.320764e+13,6.923511e+27,6.398269e+09,4.008166e+14,4.337765,16.896693,...,0.036041,0.036041,0.021776,0.000474,0.0,0.072082,-5.667055e-16,-1.204969,0.294958,11.921489
591,33253,3,2.410189e+13,1.427981e+12,5.954394e+13,3.545481e+27,6.232435e+11,1.920250e+14,2.448100,4.045118,...,0.020764,0.020764,0.013403,0.000180,0.0,0.041528,-7.817432e-17,-1.230000,0.478257,4.811490
592,33322,1,3.740236e+11,1.003053e+10,6.836876e+11,4.674287e+23,1.129324e+09,2.020392e+12,1.904838,1.851688,...,0.055444,0.055444,0.036962,0.001366,0.0,0.110887,3.833919e-16,-1.250000,0.546863,3.546271


### Merging Specifications dataset

In [28]:
# reading and converting specifications data from catorigcal to numerical
train_spes=pd.read_csv("train_specifications.csv")
train_spes = train_spes.replace(to_replace=r'Cat', value='', regex=True)
train_spes=train_spes.astype(int)
train_spes

,vehicle_id,Spec_0,Spec_1,Spec_2,Spec_3,Spec_4,Spec_5,Spec_6,Spec_7
0,0,0,0,0,0,0,0,0,0
1,2,0,1,1,0,0,0,0,1
2,3,0,1,1,1,0,0,0,1
3,4,0,0,2,1,0,0,0,1
4,5,0,2,2,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...
23545,33639,0,1,1,0,0,0,1,4
23546,33640,0,14,1,3,0,0,1,4
23547,33641,0,1,1,0,0,0,1,4
23548,33642,0,1,1,0,0,0,1,4


In [29]:
data=data.merge(train_spes,on='vehicle_id')

### Split the data

In [30]:
X = data.drop('class_label', axis=1)
y = data['class_label']
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

### Feature Selection using Logistic regression L1

In [31]:
# Initialize and train the logistic regression model with L1 penalty
model = LogisticRegression(penalty='l1', solver='liblinear', C=1.0)
model.fit(x_train, y_train)

c:\Users\adnan\anaconda3\envs\new\lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LogisticRegression(penalty='l1', solver='liblinear')

In [32]:
predictions = model.predict(x_val)
report = classification_report(y_val, predictions)
print(report)

              precision    recall  f1-score   support

           0       0.73      0.69      0.71        32
           1       0.38      0.38      0.38         8
           2       0.21      0.29      0.24        14
           3       0.59      0.53      0.56        32
           4       0.67      0.67      0.67        33

    accuracy                           0.57       119
   macro avg       0.51      0.51      0.51       119
weighted avg       0.59      0.57      0.58       119



In [33]:
# Extract coefficients
coefficients = model.coef_[0]

# Map coefficients to their feature names
feature_importance = pd.DataFrame({'Feature': x_train.columns, 'Coefficient': coefficients})

# Sort features by the absolute value of their coefficient
feature_importance['Absolute Coefficient'] = feature_importance['Coefficient'].abs()
feature_importance = feature_importance.sort_values(by='Absolute Coefficient', ascending=False)

# the feature importance
feat_import=feature_importance.loc[feature_importance["Absolute Coefficient"]<=0]

In [34]:
# drop the features that have the importance less than 0
x_train=x_train.drop(feat_import["Feature"],axis=1)
x_val=x_val.drop(feat_import["Feature"],axis=1)

### Modeling and evaluating

In [35]:
# Initialize the logistic regression model
logreg = LogisticRegression()

# Train the model
logreg.fit(x_train, y_train)

# Predict the labels of the test set
y_pred = logreg.predict(x_val)

# Calculate confusion matrix
conf_mat_log = confusion_matrix(y_val, y_pred)

report1 = classification_report(y_val, y_pred)


report = classification_report(y_val, y_pred, output_dict=True)

# Convert the report to a DataFrame
report_df_log = pd.DataFrame(report).transpose()

# Drop the support column to focus on precision, recall, and f1-score
report_df_log = report_df_log.drop('support', axis=1)

# Evaluate the model
print(report1)
print(conf_mat_log)

              precision    recall  f1-score   support

           0       0.50      0.69      0.58        32
           1       0.17      0.12      0.14         8
           2       0.09      0.07      0.08        14
           3       0.36      0.25      0.30        32
           4       0.47      0.52      0.49        33

    accuracy                           0.41       119
   macro avg       0.32      0.33      0.32       119
weighted avg       0.39      0.41      0.39       119

[[22  0  0  2  8]
 [ 1  1  1  4  1]
 [ 3  2  1  4  4]
 [ 7  3  8  8  6]
 [11  0  1  4 17]]


c:\Users\adnan\anaconda3\envs\new\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [36]:
# Initialize the Random Forest model
rf_model = RandomForestClassifier()

# Train the model
rf_model.fit(x_train, y_train)

# Predict the labels of the test set
y_pred_rf = rf_model.predict(x_val)

conf_mat_rf = confusion_matrix(y_val, y_pred_rf)
report1 = classification_report(y_val, y_pred_rf)

report = classification_report(y_val, y_pred_rf, output_dict=True)

# Convert the report to a DataFrame
report_df_rf = pd.DataFrame(report).transpose()

# Drop the support column to focus on precision, recall, and f1-score
report_df_rf = report_df_rf.drop('support', axis=1)


# Evaluate the model
print("Random Forest Classification Report:")
print(report1)
print("Random Forest Confusion Matrix:")
print(conf_mat_rf)

Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.81      0.80        32
           1       0.27      0.38      0.32         8
           2       0.40      0.29      0.33        14
           3       0.52      0.50      0.51        32
           4       0.71      0.73      0.72        33

    accuracy                           0.61       119
   macro avg       0.54      0.54      0.53       119
weighted avg       0.61      0.61      0.61       119

Random Forest Confusion Matrix:
[[26  0  0  3  3]
 [ 1  3  1  2  1]
 [ 1  2  4  6  1]
 [ 0  6  5 16  5]
 [ 5  0  0  4 24]]


In [37]:
# Initialize the Gradient Boosting model
gb_model = GradientBoostingClassifier()

# Train the model
gb_model.fit(x_train, y_train)

# Predict the labels of the test set
y_pred_gb = gb_model.predict(x_val)

conf_mat_gb = confusion_matrix(y_val, y_pred_gb)

report1 = classification_report(y_val, y_pred_gb)


report = classification_report(y_val, y_pred_gb, output_dict=True)

# Convert the report to a DataFrame
report_df_gb = pd.DataFrame(report).transpose()

# Drop the support column to focus on precision, recall, and f1-score
report_df_gb = report_df_gb.drop('support', axis=1)

# Evaluate the model
print("Gradient Boosting Classification Report:")
print(report1)
print("Gradient Boosting Confusion Matrix:")
print(conf_mat_gb)

Gradient Boosting Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.97      0.93        32
           1       0.33      0.25      0.29         8
           2       0.18      0.14      0.16        14
           3       0.65      0.62      0.63        32
           4       0.81      0.88      0.84        33

    accuracy                           0.71       119
   macro avg       0.57      0.57      0.57       119
weighted avg       0.68      0.71      0.69       119

Gradient Boosting Confusion Matrix:
[[31  0  0  1  0]
 [ 0  2  2  3  1]
 [ 0  2  2  7  3]
 [ 0  2  7 20  3]
 [ 4  0  0  0 29]]


### Evaluating the models using evaluation data

#### Reading the Evaluation data

In [38]:
validation_labels=pd.read_csv("validation_labels.csv")
validation_labels["class_label"].value_counts()

0    4910
4      76
3      30
1      16
2      14
Name: class_label, dtype: int64

In [39]:
valdation_data=pd.read_csv("validation_operational_readouts.csv")
valdation_data=pd.merge(valdation_data,validation_labels,on='vehicle_id')
valdation_data

,vehicle_id,time_step,171_0,666_0,427_0,837_0,167_0,167_1,167_2,167_3,...,397_27,397_28,397_29,397_30,397_31,397_32,397_33,397_34,397_35,class_label
0,10,3.0,46590.0,3696.0,2038959.0,1450.0,0.0,273826.0,339584.0,99834.0,...,8026.0,855.0,15.0,495.0,91665.0,169125.0,51900.0,9362.0,0.0,0
1,10,7.4,127110.0,16716.0,6501456.0,4660.0,0.0,635642.0,609742.0,288036.0,...,44312.0,7995.0,75.0,2265.0,414180.0,688891.0,258810.0,63167.0,1005.0,0
2,10,8.0,144015.0,19596.0,7327918.0,5230.0,0.0,654780.0,665756.0,300174.0,...,52203.0,9421.0,75.0,2415.0,487080.0,812071.0,300735.0,77477.0,1500.0,0
3,10,12.0,187560.0,24264.0,9286082.0,7420.0,2647.0,831628.0,794332.0,360066.0,...,61399.0,10727.0,75.0,2610.0,655620.0,1118116.0,403516.0,99587.0,1591.0,0
4,10,12.2,187575.0,24264.0,9286082.0,7420.0,NaN,NaN,NaN,NaN,...,61400.0,10727.0,75.0,2611.0,655620.0,1118116.0,403517.0,99588.0,1592.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196222,33625,67.8,1392945.0,2450.0,60748886.0,0.0,0.0,3772792.0,10954147.0,6446205.0,...,23783.0,1735.0,75.0,17827.0,219069.0,609726.0,41630.0,2284.0,60.0,0
196223,33625,73.8,1519110.0,2654.0,66374974.0,0.0,0.0,4062662.0,11906819.0,7121565.0,...,26033.0,1915.0,75.0,20227.0,247374.0,658866.0,44435.0,2480.0,60.0,0
196224,33625,77.4,1590900.0,2690.0,69656097.0,0.0,0.0,4256241.0,12550791.0,7621393.0,...,27594.0,2081.0,75.0,21307.0,263755.0,690606.0,46416.0,2585.0,60.0,0
196225,33625,83.4,1657335.0,2690.0,72538912.0,0.0,0.0,4488515.0,13016809.0,7995291.0,...,29380.0,2201.0,75.0,22462.0,285700.0,716767.0,48952.0,2810.0,60.0,0


### Selecting number of vehicles for each class

In [40]:
# Select repaired vehicles same amount as unrepaired vehicles, largest class which is 4
class_healthy = valdation_data.loc[valdation_data['class_label'] == 0]["vehicle_id"].unique()[:76]
df_0 = valdation_data.loc[(valdation_data["vehicle_id"].isin(class_healthy))]
classes_unhealthy = valdation_data.loc[valdation_data['class_label'] != 0]
valdation=pd.concat([classes_unhealthy,df_0])

### Apply PSD 

In [41]:
# Group the training data by 'vehicle_id' and apply the compute_psd function
groupedv = valdation.groupby('vehicle_id')
grouped_psd_vdfs = groupedv.apply(compute_psd)
valdation = grouped_psd_vdfs.reset_index(drop=True)

c:\Users\adnan\anaconda3\envs\new\lib\site-packages\scipy\signal\_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 102, using nperseg = 102
  warnings.warn('nperseg = {0:d} is greater than input length '
c:\Users\adnan\anaconda3\envs\new\lib\site-packages\scipy\signal\_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 20, using nperseg = 20
  warnings.warn('nperseg = {0:d} is greater than input length '
c:\Users\adnan\anaconda3\envs\new\lib\site-packages\scipy\signal\_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 53, using nperseg = 53
  warnings.warn('nperseg = {0:d} is greater than input length '
c:\Users\adnan\anaconda3\envs\new\lib\site-packages\scipy\signal\_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 35, using nperseg = 35
  warnings.warn('nperseg = {0:d} is greater than input length '
c:\Users\adnan\anaconda3\envs\new\lib\site-packages\scipy\signal\_

### Add Class Label

In [42]:
valdation=pd.merge(validation_labels,valdation,on='vehicle_id')

### Applying Feature Extraction

In [43]:
val = feature_extraction(valdation, valdation.columns[2:])
val.fillna(0,inplace=True)

C:\Users\adnan\AppData\Local\Temp\ipykernel_20464\2012294324.py:6: RuntimeWarning: invalid value encountered in divide
  psd_norm = Pxx / np.sum(Pxx)


### Merging Specifications data

In [44]:
valdation_spes=pd.read_csv("validation_specifications.csv")
valdation_spes = valdation_spes.replace(to_replace=r'Cat', value='', regex=True)
valdation_spes=valdation_spes.astype(int)
valdation_spes

,vehicle_id,Spec_0,Spec_1,Spec_2,Spec_3,Spec_4,Spec_5,Spec_6,Spec_7
0,10,0,0,0,0,0,1,0,1
1,16,0,1,1,1,0,0,0,1
2,18,0,1,1,1,0,0,0,1
3,23,0,1,1,1,0,0,0,0
4,45,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...
5041,33596,0,1,1,1,0,0,0,7
5042,33601,0,1,1,0,0,0,1,4
5043,33605,0,1,1,0,0,0,0,4
5044,33612,0,1,1,0,0,0,1,4


In [45]:
val=val.merge(valdation_spes,on='vehicle_id')
val=val.drop(feat_import["Feature"],axis=1)

In [46]:
val=pd.merge(validation_labels,val,on='vehicle_id')

### Evaluating

In [47]:
X_v = val.drop('class_label', axis=1)
y_v = val['class_label']

In [48]:
# logistic regression model
y_pred = logreg.predict(X_v)

report1 = classification_report(y_v, y_pred)
print("Logistic Regression model")
print(report1)

Logistic Regression model
              precision    recall  f1-score   support

           0       0.50      0.17      0.25        76
           1       0.00      0.00      0.00        16
           2       0.07      0.21      0.11        14
           3       0.18      0.33      0.24        30
           4       0.33      0.34      0.33        76

    accuracy                           0.25       212
   macro avg       0.22      0.21      0.19       212
weighted avg       0.33      0.25      0.25       212



In [49]:
# random forest model
vy_pred_rf = rf_model.predict(X_v)

conf_mat_rf = confusion_matrix(y_v, vy_pred_rf)
rreport1 = classification_report(y_v, vy_pred_rf)
# Evaluate the model
print("Random Forest Classification Report:")
print(rreport1)
print("Random Forest Confusion Matrix:")
print(conf_mat_rf)

Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.59      0.68        76
           1       0.00      0.00      0.00        16
           2       0.04      0.07      0.05        14
           3       0.15      0.50      0.24        30
           4       0.50      0.16      0.24        76

    accuracy                           0.34       212
   macro avg       0.30      0.26      0.24       212
weighted avg       0.49      0.34      0.37       212

Random Forest Confusion Matrix:
[[45  2  2 25  2]
 [ 2  0  3 10  1]
 [ 2  0  1  6  5]
 [ 1  2  8 15  4]
 [ 7  2 14 41 12]]


In [50]:
# Gradient Boosting model
vy_pred_gb = gb_model.predict(X_v)

conf_mat_gb = confusion_matrix(y_v, vy_pred_gb)
greport1 = classification_report(y_v, vy_pred_gb)
# Evaluate the model
print("Gradient Boosting model")
print(greport1)
print(conf_mat_gb)

Gradient Boosting model
              precision    recall  f1-score   support

           0       0.98      0.67      0.80        76
           1       0.00      0.00      0.00        16
           2       0.08      0.14      0.10        14
           3       0.22      0.80      0.35        30
           4       0.19      0.04      0.07        76

    accuracy                           0.38       212
   macro avg       0.29      0.33      0.26       212
weighted avg       0.46      0.38      0.36       212

[[51  2  7  4 12]
 [ 0  0  3 13  0]
 [ 0  0  2 11  1]
 [ 0  4  2 24  0]
 [ 1  4 11 57  3]]
